In [1]:
import numpy as np
import pandas as pd
import os
import sys
import argparse
import valencia_predictor
from importlib import reload
reload(valencia_predictor)
from valencia_predictor import ValenciaPredictor
from valencia_predictor import ModelPreset
import warnings
warnings.filterwarnings("ignore") # Let's get rid of those warnings
import pandas as pd
from sklearn.metrics import mean_absolute_error


In [2]:
# Vamos a hacer una lista que tenga fechas desde el 1 de enero de 2021 hasta el 31 de julio de 2021
# donde cada elemento de la lista es una tupla con el inicio y el fin del mes
#fechas_prediccion = []
#for i in range(1, 8):
    # Hay que tener cuidado con los meses que tienen 31 días y los que tienen 30
#    if i in [1, 3, 5, 7]:
#        fecha_inicio = f"2021-0{i}-01"
#        fecha_fin = f"2021-0{i}-31"
#    elif i in [4, 6]:
#        fecha_inicio = f"2021-0{i}-01"
#        fecha_fin = f"2021-0{i}-30"
#    else:
#        fecha_inicio = f"2021-0{i}-01"
#        fecha_fin = f"2021-0{i}-28"
#    fechas_prediccion.append((fecha_inicio, fecha_fin))

#fechas_prediccion = [("2020-12-28","2021-1-11"),("2021-1-11","2021-1-25"),("2021-1-25","2021-2-8"),("2021-2-8","2021-2-22"),("2021-2-22","2021-3-8"),
#                    ("2021-3-8","2021-3-22"),("2021-3-22","2021-4-5"),("2021-4-5","2021-4-19"),("2021-4-19","2021-5-3"),
#                    ("2021-5-3","2021-5-17"),("2021-5-17","2021-5-31"),("2021-5-31","2021-6-14"),("2021-6-14","2021-6-28"),
#                    ("2021-6-28","2021-7-12"),("2021-7-12","2021-7-26"),("2021-7-26","2021-8-9")]
#fechas_entrenamiento = [("2020-09-01", "2021-03-31")]
#fechas_entrenamiento = [("2020-09-01", "2020-12-31"), ("2020-09-01", "2021-01-31"), ("2020-09-01", "2021-02-28"), 
#                        ("2020-09-01", "2021-03-31"), ("2020-09-01", "2021-04-30"), ("2020-09-01", "2021-05-31"), 
#                        ("2020-09-01", "2021-06-30")]
#fechas_entrenamiento = [("2020-09-01", "2020-12-27"), ("2020-09-01", "2021-1-10"), ("2020-09-01", "2021-1-24"), ("2020-09-01", "2021-2-7"),
#                       ("2020-09-01", "2021-2-21"), ("2020-09-01", "2021-3-7"), ("2020-09-01", "2021-3-21"), ("2020-09-01", "2021-4-4"), ("2020-09-01", "2021-4-18"),
#                       ("2020-09-01", "2021-5-2"), ("2020-09-01", "2021-5-16"), ("2020-09-01", "2021-5-30"), ("2020-09-01", "2021-6-13"), ("2020-09-01", "2021-6-27"),
#                       ("2020-09-01", "2021-7-11"), ("2020-09-01", "2021-7-25")]

fechas_prediccion = [("2020-12-28","2021-1-11"),("2021-1-11","2021-1-25"),("2021-1-25","2021-2-8"),("2021-2-8","2021-2-22"),("2021-2-22","2021-3-8"),("2021-3-8","2021-3-22"),("2021-3-22","2021-4-5"),("2021-4-5","2021-4-19"),("2021-4-19","2021-5-3"),
("2021-5-3","2021-5-17"),("2021-5-17","2021-5-31"),("2021-5-31","2021-6-14"),("2021-6-14","2021-6-28"),
("2021-6-28","2021-7-12"),("2021-7-12","2021-7-26"),("2021-7-26","2021-8-9")]

fechas_entrenamiento = [("2020-09-01", "2020-12-27"), ("2020-09-01", "2021-1-10"), ("2020-09-01", "2021-1-24"), 
                        ("2020-09-01", "2021-2-7"), ("2020-09-01", "2021-2-21"), ("2020-09-01", "2021-3-7"), 
                        ("2020-09-01", "2021-3-21"), ("2020-09-01", "2021-4-4"), ("2020-09-01", "2021-4-18"),
                        ("2020-09-01", "2021-5-2"), ("2020-09-01", "2021-5-16"), ("2020-09-01", "2021-5-30"), 
                        ("2020-09-01", "2021-6-13"), ("2020-09-01", "2021-6-27"), ("2020-09-01", "2021-7-11"), 
                        ("2020-09-01", "2021-7-25")]
                        
                        
                        
#                        ("2020-09-01", "2021-2-14"), ("2020-09-01", "2021-3-14"), 
#                        ("2020-09-01", "2021-4-11"), ("2020-09-01", "2021-5-09"), ("2020-09-01", "2021-6-13"), 
#                        ("2020-09-01", "2021-7-11")]
# Ahora vamos a realizar 100 entrenamientos, uno por cada mes
df = pd.read_csv('data/OxCGRT_latest.csv')

#training_geos = ["United States", "Portugal", "United Kingdom", "Italy", "Spain", "Germany", "Israel", "France", "Greece", "Hungary"]

training_geos = ["Germany", "Spain", "United Kingdom", "Italy", "France",
                 "Poland", "Romania", "Netherlands", "Belgium", "Greece",
                 "Czech Republic", "Sweden", "Portugal", "Austria", "Denmark",
                 "Bulgaria", "Finland", "Norway", "Ireland", "Slovenia",
                 "Croatia", "Lithuania", "United States", "Canada", "Mexico", "Brazil", "Argentina",
                 "Chile", "Colombia", "Peru", "Venezuela", "Paraguay",
                 "Nicaragua", "Guatemala", "Ecuador", "Bolivia", "Cuba",
                 "Haiti", "Dominican Republic", "El Salvador", "Honduras",
                 "Costa Rica"]

#train_preset = ModelPreset.VAC_H7_SUS
LATEST_DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_nat_latest.csv'
GEO_FILE = "data/countries_regions.csv"
DATA_FILE = 'data/OxCGRT_latest.csv'
# Creamos un dataframe que tenga el mae, el mes y el trial
df_mae = pd.DataFrame(columns=['mae', 'trial', 'mes'])
for i in range(1,10):
    print("Trial: ", i)
    for j in range(len(fechas_entrenamiento)):
        print("Entrenando con el mes: ", fechas_entrenamiento[j])
        print("el j es" ,j)
        start_date_str = fechas_entrenamiento[j][0]
        end_date_str = fechas_entrenamiento[j][1]
    #   # path_to_model = f"model_23_03_23/valencia_model_weights_"+str(i)+"_"+str(j)+".h5" # La i indica el trial y la j el mes
        path_to_model = f"none/valencia_model_weights_"+str(i)+"_"+str(j)+".h5" # La i indica el trial y la j el mes

        #predictor = ValenciaPredictor(model_preset=train_preset)
        predictor = ValenciaPredictor()
        predictor_model = predictor.train(geos=training_geos, start_date_str=start_date_str, end_date_str=end_date_str) #, min_vaccination_percentage=min_vaccination)

        print('Saving model to', path_to_model)
        predictor_model.save_weights(path_to_model)
        print("Testeando con el mes: ", fechas_prediccion[j])
        pred_start_date_str = fechas_prediccion[j][0]
        pred_end_date_str = fechas_prediccion[j][1]
        pred_start_date = pd.to_datetime(pred_start_date_str)
        pred_end_date = pd.to_datetime(pred_end_date_str)
        preds_df = predictor.predict_df(pred_start_date, pred_end_date, LATEST_DATA_URL)
        # Obtenemos el mae de la predicción
        df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
        #ground_truth_df = df[df['CountryName'].isin(training_geos)]
        ground_truth_df = df
        ground_truth_df = ground_truth_df[ground_truth_df['Date'] >= pred_start_date]
        ground_truth_df = ground_truth_df[ground_truth_df['Date'] <= pred_end_date]
        ground_truth_df = ground_truth_df[['CountryName', 'Date', 'ConfirmedCases']]
        # Creamos una nueva columna, que tenga el valor de la columna 'ConfirmedCases' respecto al día anterior
        ground_truth_df['ConfirmedCases_Diff'] = ground_truth_df.groupby('CountryName')['ConfirmedCases'].diff()
        #print("Number of rows in ground truth: {}".format(len(ground_truth_df)))
        # Place 0 in NaNs
        ground_truth_df['ConfirmedCases_Diff'] = ground_truth_df['ConfirmedCases_Diff'].fillna(0)
        # Cogemos las predicciones de los países que nos interesan
        #pred = preds_df[preds_df['CountryName'].isin(training_geos)]
        pred = preds_df
        y_true = ground_truth_df['ConfirmedCases_Diff']
        y_pred = pred['PredictedDailyNewCases']
        mae = mean_absolute_error(y_true, y_pred).round(2)
        print("MAE: ", mae)
        
        # Guardamos las predicciones
   #     #preds_df.to_csv("predictions/23_03_23/robojudge_test_"+str(i)+"_"+str(j)+".csv", index=False)
        preds_df.to_csv("none/robojudge_test_"+str(i)+"_"+str(j)+".csv", index=False)
        # Guardamos el mae, el mes y el trial en un dataframe
        df_mae = df_mae.append({'mae': mae, 'trial': i, 'mes': j}, ignore_index=True)
        del predictor # Borramos el predictor para que no se acumulen en memoria
        print("Guardando el dataframe con los mae")
   #     #df_mae.to_csv("maes_23.csv", index=False)
        df_mae.to_csv("none/maes_none.csv", index=False)

Trial:  1
Entrenando con el mes:  ('2020-09-01', '2020-12-27')
el j es 0
ValenciaPredictor ready
Creating numpy arrays for Keras for each country...
Columnas del entrenamiento: Index(['CountryName', 'RegionName', 'GeoID', 'Date', 'ConfirmedCases',
       'ConfirmedDeaths', 'Population', 'ProportionImmunized',
       'C1M_School closing', 'C2M_Workplace closing',
       'C3M_Cancel public events', 'C4M_Restrictions on gatherings',
       'C5M_Close public transport', 'C6M_Stay at home requirements',
       'C7M_Restrictions on internal movement',
       'C8EV_International travel controls', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing', 'H6M_Facial Coverings',
       'NewCases', 'NewDeaths', 'SmoothNewCases', 'SmoothNewDeaths',
       'recuperados', 'kk', 'VacunadosDiarios', 'DiezPorCiento',
       'TreintaCincoPorCiento', 'SesentaYCincoPorCiento', 'CaseRatio',
       'DeathRatio', 'ProportionInfected', 'PredictionRatio',
       'PredictionRatioVac'

183/183 [==============================] - 0s 1ms/step - loss: 0.0473
Train Loss: 0.04671444743871689
Val Loss: 0.06383105367422104
Test Loss: 0.04727381467819214
Done
Saving model to none/valencia_model_weights_1_4.h5
Testeando con el mes:  ('2021-2-22', '2021-3-8')
MAE:  1598.18
Guardando el dataframe con los mae
Entrenando con el mes:  ('2020-09-01', '2021-3-7')
el j es 5
ValenciaPredictor ready
Creating numpy arrays for Keras for each country...
Columnas del entrenamiento: Index(['CountryName', 'RegionName', 'GeoID', 'Date', 'ConfirmedCases',
       'ConfirmedDeaths', 'Population', 'ProportionImmunized',
       'C1M_School closing', 'C2M_Workplace closing',
       'C3M_Cancel public events', 'C4M_Restrictions on gatherings',
       'C5M_Close public transport', 'C6M_Stay at home requirements',
       'C7M_Restrictions on internal movement',
       'C8EV_International travel controls', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing', 'H6M_Facial C

275/275 [==============================] - 0s 1ms/step - loss: 0.0405
Train Loss: 0.038927413523197174
Val Loss: 0.06118113175034523
Test Loss: 0.04054144024848938
Done
Saving model to none/valencia_model_weights_1_9.h5
Testeando con el mes:  ('2021-5-3', '2021-5-17')
MAE:  2888.83
Guardando el dataframe con los mae
Entrenando con el mes:  ('2020-09-01', '2021-5-16')
el j es 10
ValenciaPredictor ready
Creating numpy arrays for Keras for each country...
Columnas del entrenamiento: Index(['CountryName', 'RegionName', 'GeoID', 'Date', 'ConfirmedCases',
       'ConfirmedDeaths', 'Population', 'ProportionImmunized',
       'C1M_School closing', 'C2M_Workplace closing',
       'C3M_Cancel public events', 'C4M_Restrictions on gatherings',
       'C5M_Close public transport', 'C6M_Stay at home requirements',
       'C7M_Restrictions on internal movement',
       'C8EV_International travel controls', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing', 'H6M_Facia

367/367 [==============================] - 0s 1ms/step - loss: 0.0414
Train Loss: 0.03917314112186432
Val Loss: 0.06777146458625793
Test Loss: 0.0414302833378315
Done
Saving model to none/valencia_model_weights_1_14.h5
Testeando con el mes:  ('2021-7-12', '2021-7-26')
MAE:  1972.85
Guardando el dataframe con los mae
Entrenando con el mes:  ('2020-09-01', '2021-7-25')
el j es 15
ValenciaPredictor ready
Creating numpy arrays for Keras for each country...
Columnas del entrenamiento: Index(['CountryName', 'RegionName', 'GeoID', 'Date', 'ConfirmedCases',
       'ConfirmedDeaths', 'Population', 'ProportionImmunized',
       'C1M_School closing', 'C2M_Workplace closing',
       'C3M_Cancel public events', 'C4M_Restrictions on gatherings',
       'C5M_Close public transport', 'C6M_Stay at home requirements',
       'C7M_Restrictions on internal movement',
       'C8EV_International travel controls', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing', 'H6M_Facia

165/165 [==============================] - 0s 1ms/step - loss: 0.0473
Train Loss: 0.04595170170068741
Val Loss: 0.06828413903713226
Test Loss: 0.04725824296474457
Done
Saving model to none/valencia_model_weights_2_3.h5
Testeando con el mes:  ('2021-2-8', '2021-2-22')
MAE:  1112.36
Guardando el dataframe con los mae
Entrenando con el mes:  ('2020-09-01', '2021-2-21')
el j es 4
ValenciaPredictor ready
Creating numpy arrays for Keras for each country...
Columnas del entrenamiento: Index(['CountryName', 'RegionName', 'GeoID', 'Date', 'ConfirmedCases',
       'ConfirmedDeaths', 'Population', 'ProportionImmunized',
       'C1M_School closing', 'C2M_Workplace closing',
       'C3M_Cancel public events', 'C4M_Restrictions on gatherings',
       'C5M_Close public transport', 'C6M_Stay at home requirements',
       'C7M_Restrictions on internal movement',
       'C8EV_International travel controls', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing', 'H6M_Facial 

...

293/293 [==============================] - 0s 1ms/step - loss: 0.0396
Train Loss: 0.038261186331510544
Val Loss: 0.06236119195818901
Test Loss: 0.03959108516573906
Done
Saving model to none/valencia_model_weights_5_10.h5
Testeando con el mes:  ('2021-5-17', '2021-5-31')
MAE:  1271.3
Guardando el dataframe con los mae
Entrenando con el mes:  ('2020-09-01', '2021-5-30')
el j es 11
ValenciaPredictor ready
Creating numpy arrays for Keras for each country...
Columnas del entrenamiento: Index(['CountryName', 'RegionName', 'GeoID', 'Date', 'ConfirmedCases',
       'ConfirmedDeaths', 'Population', 'ProportionImmunized',
       'C1M_School closing', 'C2M_Workplace closing',
       'C3M_Cancel public events', 'C4M_Restrictions on gatherings',
       'C5M_Close public transport', 'C6M_Stay at home requirements',
       'C7M_Restrictions on internal movement',
       'C8EV_International travel controls', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing', 'H6M_Faci

385/385 [==============================] - 0s 1ms/step - loss: 0.0447
Train Loss: 0.043438199907541275
Val Loss: 0.07198713719844818
Test Loss: 0.044698797166347504
Done
Saving model to none/valencia_model_weights_5_15.h5
Testeando con el mes:  ('2021-7-26', '2021-8-9')
MAE:  2721.92
Guardando el dataframe con los mae
Trial:  6
Entrenando con el mes:  ('2020-09-01', '2020-12-27')
el j es 0
ValenciaPredictor ready
Creating numpy arrays for Keras for each country...
Columnas del entrenamiento: Index(['CountryName', 'RegionName', 'GeoID', 'Date', 'ConfirmedCases',
       'ConfirmedDeaths', 'Population', 'ProportionImmunized',
       'C1M_School closing', 'C2M_Workplace closing',
       'C3M_Cancel public events', 'C4M_Restrictions on gatherings',
       'C5M_Close public transport', 'C6M_Stay at home requirements',
       'C7M_Restrictions on internal movement',
       'C8EV_International travel controls', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing'

183/183 [==============================] - 0s 1ms/step - loss: 0.0478
Train Loss: 0.04554064944386482
Val Loss: 0.06468269228935242
Test Loss: 0.04781431704759598
Done
Saving model to none/valencia_model_weights_6_4.h5
Testeando con el mes:  ('2021-2-22', '2021-3-8')
MAE:  1682.64
Guardando el dataframe con los mae
Entrenando con el mes:  ('2020-09-01', '2021-3-7')
el j es 5
ValenciaPredictor ready
Creating numpy arrays for Keras for each country...
Columnas del entrenamiento: Index(['CountryName', 'RegionName', 'GeoID', 'Date', 'ConfirmedCases',
       'ConfirmedDeaths', 'Population', 'ProportionImmunized',
       'C1M_School closing', 'C2M_Workplace closing',
       'C3M_Cancel public events', 'C4M_Restrictions on gatherings',
       'C5M_Close public transport', 'C6M_Stay at home requirements',
       'C7M_Restrictions on internal movement',
       'C8EV_International travel controls', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing', 'H6M_Facial C

275/275 [==============================] - 0s 1ms/step - loss: 0.0414
Train Loss: 0.040233224630355835
Val Loss: 0.062859445810318
Test Loss: 0.04138932004570961
Done
Saving model to none/valencia_model_weights_6_9.h5
Testeando con el mes:  ('2021-5-3', '2021-5-17')
MAE:  1680.61
Guardando el dataframe con los mae
Entrenando con el mes:  ('2020-09-01', '2021-5-16')
el j es 10
ValenciaPredictor ready
Creating numpy arrays for Keras for each country...
Columnas del entrenamiento: Index(['CountryName', 'RegionName', 'GeoID', 'Date', 'ConfirmedCases',
       'ConfirmedDeaths', 'Population', 'ProportionImmunized',
       'C1M_School closing', 'C2M_Workplace closing',
       'C3M_Cancel public events', 'C4M_Restrictions on gatherings',
       'C5M_Close public transport', 'C6M_Stay at home requirements',
       'C7M_Restrictions on internal movement',
       'C8EV_International travel controls', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing', 'H6M_Facial 

KeyboardInterrupt: 

In [ ]:
len(fechas_entrenamiento)

In [ ]:
j

In [18]:
min(df_mae[df_mae.mes==15.0]["mae"])

2358.29

for i in range(1,3):
    print(i)

In [ ]:
df_mae = pd.read_csv("maes_23.csv")